In [36]:
import pandas
from itertools import product
import pickle

In [37]:
_exec_scenarios = 52

In [38]:
def create_list(n_total: int, n: int):
    return [1] * n + [0] * (n_total - n)

In [39]:
def create_event_dict(n_total: int):
    default_list = [1] * n_total

    # If event names are same before '_'; they are considered mutually exclusive
    event_dict = {
        'cap2_1': {'prob': 0.06, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): create_list(n_total, 13)})},
        'cap2_2': {'prob': 0.12, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): create_list(n_total, 26)})},
        'cap2_3': {'prob': 0.17, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): create_list(n_total, 39)})},
        'cap2_nd': {'prob': 0.65, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): default_list})},

        # 'cap4_1': {'prob': 0.05, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): create_list(n_total, 1)})},
        # 'cap4_2': {'prob': 0.15, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): create_list(n_total, 2)})},
        # 'cap4_3': {'prob': 0.18, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): create_list(n_total, 3)})},
        # 'cap4_nd': {'prob': 0.62, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): default_list})},

        # 'cap7_1': {'prob': 0.04, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(n_total, 1)})},
        # 'cap7_2': {'prob': 0.15, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(n_total, 2)})},
        # 'cap7_3': {'prob': 0.2, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(n_total, 3)})},
        # 'cap7_nd': {'prob': 0.61, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): default_list})},

        # 'res1_1': {'prob': 0.06, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(n_total, 1)})},
        # 'res1_2': {'prob': 0.12, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(n_total, 2)})},
        # 'res1_3': {'prob': 0.19, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(n_total, 3)})},
        # 'res1_nd': {'prob': 0.63, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): default_list})},

        # 'res6_1': {'prob': 0.03, 'factor': pandas.DataFrame(data={('loc6', 'com1_pur'): create_list(n_total, 13)})},
        # 'res6_2': {'prob': 0.16, 'factor': pandas.DataFrame(data={('loc6', 'com1_pur'): create_list(n_total, 26)})},
        # 'res6_3': {'prob': 0.17, 'factor': pandas.DataFrame(data={('loc6', 'com1_pur'): create_list(n_total, 39)})},
        # 'res6_nd': {'prob': 0.64, 'factor': pandas.DataFrame(data={('loc6', 'com1_pur'): default_list})},
        # 
        # 'truck47_2': {'prob': 0.2, 'factor': pandas.DataFrame(data={('truck47', 'com1_loc4_out'): create_list(n_total, 26)})},
        # 'truck47_nd': {'prob': 0.8, 'factor': pandas.DataFrame(data={('truck47', 'com1_loc4_out'): default_list})},
        # 
        # 'truck25_1': {'prob': 0.3, 'factor': pandas.DataFrame(data={('truck25', 'com1_loc2_out'): create_list(n_total, 13)})},
        # 'truck25_nd': {'prob': 0.7, 'factor': pandas.DataFrame(data={('truck25', 'com1_loc2_out'): default_list})},
    }

    return event_dict

In [40]:
# Function to generate the scenario dictionary for n sets of events
def create_scenario_dict(event_dict):
    # Extract unique event prefixes (e.g., 'cap2', 'cap4', ...)
    event_prefixes = set(key.split('_')[0] for key in event_dict)

    # Group events by their prefixes
    grouped_events = {prefix: [key for key in event_dict if key.startswith(prefix)] for prefix in event_prefixes}

    # Create all possible combinations of events across the different groups
    event_combinations = list(product(*grouped_events.values()))

    scenario_dict = {}

    # Iterate over all event combinations
    for combination in event_combinations:
        # Construct the scenario key
        scenario_key = ' '.join(combination)

        # Calculate the probability of this scenario
        prob = 1
        combined_factor = None

        for event_key in combination:
            # Multiply probabilities
            prob *= event_dict[event_key]['prob']

            # Combine factors (assumes they are pandas DataFrames)
            if combined_factor is None:
                combined_factor = event_dict[event_key]['factor'].copy()
            else:
                combined_factor = combined_factor.add(event_dict[event_key]['factor'], fill_value=0)

        # Add to the scenario dictionary
        scenario_dict[scenario_key] = {'prob': prob, 'factor': combined_factor}

    return scenario_dict

In [41]:
event_dict = create_event_dict(n_total=_exec_scenarios)
scenario_dict = create_scenario_dict(event_dict=event_dict)
scenario_names = list(scenario_dict.keys())

with open('backlog_scen_dict.pkl', 'wb') as file:
    pickle.dump(scenario_dict, file)

# print(*scenario_names, sep='\n')
print(f"Sum of probabilities of all scenarios: {sum(scenario_dict[scen]['prob'] for scen in scenario_dict):.6f}")
print(f'Number of considered scenarios: {len(scenario_names)}')

Sum of probabilities of all scenarios: 1.000000
Number of considered scenarios: 4


In [42]:
with open('backlog_scen_dict.pkl', 'rb') as file:
    load_scenario_dict = pickle.load(file)
load_scenario_names = list(load_scenario_dict.keys())

print(f"Sum of probabilities of all scenarios: {sum(load_scenario_dict[scen]['prob'] for scen in load_scenario_dict):.6f}")
print(f'Number of considered scenarios: {len(load_scenario_names)}')

Sum of probabilities of all scenarios: 1.000000
Number of considered scenarios: 4
